In [1]:
import os
import pandas as pd
import numpy as np
import math
from datetime import datetime
import datetime as dt
import tensorflow as tf
from tensorflow import keras
from keras.layers import GRU, Dropout, SimpleRNN, LSTM, Dense, SimpleRNN, GRU
from keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn

import plotly as py
import plotly.io as pio

# Mute sklearn warnings
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
simplefilter(action='ignore', category=DeprecationWarning)

from plotly.offline import init_notebook_mode

# Show charts when running kernel
init_notebook_mode(connected=True)

from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score 
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, accuracy_score
from sklearn.preprocessing import MinMaxScaler

from itertools import cycle
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import seaborn as sns
import matplotlib.pyplot as plt
from colorama import Fore

In [2]:
# Path to your CSV file
csv_file_path = '/Users/michael/Desktop/annual_generation_state.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(csv_file_path)

# for col in df.columns:
#     print(col)

# Display the first few rows of the DataFrame
# print(df.head())

# Convert 'GENERATION (Megawatthours)' to numeric format, if it contains commas
df['GENERATION (Megawatthours)'] = df['GENERATION (Megawatthours)'].str.replace(',', '').astype(float)

# Filter for the year 2021
df_2021 = df[df['YEAR'] == 2021]

df_2021 = df_2021[df_2021['TYPE OF PRODUCER'] == 'Total Electric Power Industry']

# Define renewable energy sources based on your update
renewable_sources = [
    'Wind', 
    'Wood and Wood Derived Fuels', 
    'Other Biomass', 
    'Hydroelectric Conventional', 
    'Solar Thermal and Photovoltaic', 
    'Pumped Storage', 
    'Geothermal'
]

# Identify renewable energy generation
df_2021['Renewable'] = df_2021['ENERGY SOURCE'].isin(renewable_sources)

# Group by State and Energy Source, then sum the generation
state_energy_generation = df_2021.groupby(['STATE', 'ENERGY SOURCE', 'Renewable'])['GENERATION (Megawatthours)'].sum().reset_index()

# Calculate total and renewable generation by state
total_generation_by_state = df_2021.groupby('STATE')['GENERATION (Megawatthours)'].sum() / 2
renewable_generation_by_state = df_2021[df_2021['Renewable']].groupby('STATE')['GENERATION (Megawatthours)'].sum()

# Calculate the percentage of renewable generation for each state
percentage_renewable = (renewable_generation_by_state / total_generation_by_state * 100).reset_index(name='Percentage Renewable')

# Merge the percentage data back with the detailed state and energy source data
detailed_state_data = pd.merge(state_energy_generation, percentage_renewable, on='STATE')

# Now, print out the details for each state
for state in detailed_state_data['STATE'].unique():
    print(f"State: {state}")
    state_data = detailed_state_data[detailed_state_data['STATE'] == state]
    for _, row in state_data.iterrows():
        print(f"  Energy Source: {row['ENERGY SOURCE']}, Renewable: {row['Renewable']}, Generation (MWh): {row['GENERATION (Megawatthours)']}")
    state_percentage_renewable = state_data['Percentage Renewable'].iloc[0]
    print(f"  Percentage of Renewable Energy: {state_percentage_renewable:.2f}%")
    print("")


State: AK
  Energy Source: Coal, Renewable: False, Generation (MWh): 752894.0
  Energy Source: Hydroelectric Conventional, Renewable: True, Generation (MWh): 1688900.0
  Energy Source: Natural Gas, Renewable: False, Generation (MWh): 3085396.0
  Energy Source: Other, Renewable: False, Generation (MWh): -4141.0
  Energy Source: Other Biomass, Renewable: True, Generation (MWh): 38071.0
  Energy Source: Petroleum, Renewable: False, Generation (MWh): 902486.0
  Energy Source: Solar Thermal and Photovoltaic, Renewable: True, Generation (MWh): 0.0
  Energy Source: Total, Renewable: False, Generation (MWh): 6595818.0
  Energy Source: Wind, Renewable: True, Generation (MWh): 132212.0
  Energy Source: Wood and Wood Derived Fuels, Renewable: True, Generation (MWh): 0.0
  Percentage of Renewable Energy: 28.19%

State: AL
  Energy Source: Coal, Renewable: False, Generation (MWh): 26900909.0
  Energy Source: Hydroelectric Conventional, Renewable: True, Generation (MWh): 11520809.0
  Energy Source: 

In [3]:
import plotly.graph_objects as go

import pandas as pd
df = pd.read_csv('/Users/michael/Desktop/us_states_renewable_energy.csv')

# Convert the 'State' column to string type for consistency
df['State'] = df['State'].astype(str)

# Create the choropleth map
fig = go.Figure(data=go.Choropleth(
    locations=df['State'],  # Use the 'State' column for state codes
    z=df['Percentage Renewable'].astype(float),  # Use the 'Percentage Renewable' column for the color scale
    locationmode='USA-states',  # Set the location mode to 'USA-states'
    colorscale='Greens',  # Changed the colorscale to 'Greens' for a more thematic fit
    autocolorscale=False,
    zmin=0,  # Set the minimum value of the color scale
    zmax=100,  # Set the maximum value of the color scale
    hoverinfo='none',  # This part is new
    marker_line_color='black',  # Line markers between states
    marker_line_width=0.5,  # Outline thickness
    colorbar_title="Percentage"
))

# Update the layout for the new data
fig.update_layout(
    geo=dict(
        scope='usa',  # Limit map scope to USA
        projection=go.layout.geo.Projection(type='albers usa'),  # Use Albers projection
        showlakes=True,  # Show lakes
        lakecolor='rgb(255, 255, 255)'  # Set lake color
    ),
    autosize=True,
    width=1000,
    height=500,
    font=dict(
        family="Times New Roman",
        size=18,
        color="Black"
    ),
    margin=dict(  # This part is new
        l=10,  # Left margin
        r=10,  # Right margin
        b=10,  # Bottom margin
        t=10,  # Top margin
        pad=0  # Padding
    )
)

# Show the figure
fig.show()